# Funkcijsko programiranje 1

Funkcijsko programiranje je stil programiranja, v katerem programiramo z "zlaganjem" funkcij. Funkcije ne smejo imeti stranskih učinkov, kot je spreminjanje globalnih spremenljivk ali lastnih argumentov. Če pokličemo funkcijo z enakimi argumenti, mora vedno vrniti enak rezultat. Podatkovne strukture so nespremenljive (*immutable*), zato jih ne moremo spreminjati, temveč sestavljamo nove. Ne uporabljamo zank, ki spreminjajo kakšne lokalne ali globalne števce, ampak uporabljamo rekurzijo.

Način programiranja je na prvi pogled precej omejujoč, vendar ima svoje prednosti. Prevajalniki oz. interpreterji imajo v tako omejenih programih več možnosti za optimizacije. Lažje je tudi formalno dokazovanje pravilnosti takih programov. Z vidika učinkovitosti nam tako programiranje omogoča enostavno paralelizacijo, kar je pogosto vse prej kot trivialno.

Med funkcijske programske jezike uvrščamo Haskell, Racket, Lisp, ... Čeprav ne boste programirali v funkcijskem jeziku, pa vam pridejo prav osvojeni pristopi pri reševanju problemov v drugih jeziki. Skoraj vsi programski jeziki namreč ponujajo konstrukte, ki omogočajo funkcijski način programiranja (Python, C++, Java, Kotlin, Go, ...)

## Funkcije višjega reda

Funkcije višjega reda so tiste funkcije, ki sprejmejo in/ali vračajo druge funkcije. Primer take funkcije v matematiki je odvod. Najprej si oglejmo primer funkcij, ki kot argument sprejmejo drugo funkcijo. Vračanju funkcij pa se posvetimo na naslednjih predavanjih.

Napišimo program, ki bo v seznamu imen poiskal prvega po abecedi ne glede na velikost črk. Nato napišimo še podoben program, ki bo poisal najdaljše ime v seznamu.

In [1]:
imena = ["andreja", "Bine", "Cene", "Dominika", "Eva"]
prvi = imena[0]
for ime in imena:
    if ime.lower() < prvi.lower():
        prvi = ime
print("prvi po abecedi:", prvi)

prvi = imena[0]
for ime in imena:
    if len(ime) > len(prvi):
        prvi = ime
print("najdaljsi:", prvi)

prvi po abecedi: andreja
najdaljsi: Dominika


Vidimo, da je logika programa v obeh primerih precej podobna. Po vrsti obravnavamo elemente seznama, jih primerjamo s trenutno najboljšim in ga spremenimo, če je treba. Programa se razlikujeta samo v pogoju, kdaj je nek element boljši od drugega. Obe rešitvi lahko posplošimo s funkcijo višjega reda `najboljsi`, ki poleg seznama `imena` sprejme tudi funkcijo `boljsi`, ki bo znala določiti, ali je prvi element boljši od drugega.

In [2]:
def najboljsi(imena, boljsi):
    prvi = imena[0]
    for ime in imena:
        if boljsi(ime, prvi):
            prvi = ime
    return prvi

Sedaj lahko s pripravo različnih primerjalnih funkcij hitro sprogramiramo iskanje prvega elementa po nekem kriteriju. Pazljivi moramo biti pri podajanju primerjalne funkcije kot argumenta. Klic `najboljsi(imena, manjsi_leksikografsko()))` bi namreč poskusil izvesti funkcijo `manjsi_leksikografsko` in kot argument funkciji `najboljsi` podal njen rezultat. Tega pa ne želimo, temveč bi radi podali kot argument funkcijo, ne njenega rezultata. To storimo tako, da podamo samo ime funkcije `najboljsi(imena, manjsi_leksikografsko))` in pri tem izpustimo oklepaje, ki označujejo klic funkcije.

In [3]:
def manjsi_leksikografsko(x, y):
    return x.lower() < y.lower()

def manjsi_dolzina(x, y):
    return len(x) < len(y)

def vecji_dolzina(x, y):
    return len(x) > len(y)

print("leksikografsko:", najboljsi(imena, manjsi_leksikografsko))
print("najkrajsi:", najboljsi(imena, manjsi_dolzina))
print("najdaljsi:", najboljsi(imena, vecji_dolzina))

leksikografsko: andreja
najkrajsi: Eva
najdaljsi: Dominika


## Anonimne funkcije (lambda)

Primerjalne funkcije smo definirali z določenimi imeni in jih nato uporabili samo enkrat. Prikladno bi bilo, če bi lahko definicijo funkcije napisali kar na mestu, kjer želimo to funkcije uporabiti. To lahko storimo z anonimnimi (*lambda*) funkcijami. Z njimi definiramo funkcijo, vendar ji ne priredimo imena. Lambda funkcije so sestavljene iz enega samega izraza, ki definira vrnjeno vrednost. Pogosto jih uporabljamo s standardnimi funkcijami kot so `min`, `max`, `sort`, `sorted`, ki kot imenovan argument `key` sprejmejo funkcijo.

In [4]:
print("najkrajsi (najboljsi):", najboljsi(imena, lambda x, y: len(x) < len(y)))
print("najdaljsi (max):", max(imena, key=len))
print("po abecedi:", sorted(imena))
print("po dolzini:", sorted(imena, key=len))
print("case-insensitive:", sorted(imena, key=lambda x: x.lower()))

najkrajsi (najboljsi): Eva
najdaljsi (max): Dominika
po abecedi: ['Bine', 'Cene', 'Dominika', 'Eva', 'andreja']
po dolzini: ['Eva', 'Bine', 'Cene', 'andreja', 'Dominika']
case-insensitive: ['andreja', 'Bine', 'Cene', 'Dominika', 'Eva']


## Map, filter, reduce

Funkcije `map`, `filter` in `reduce` (v drugih jezikih imenovan `fold`) so poleg rekurzije glavna orodja funkcijskega programiranja.

**Map** sprejme funkcijo in seznam ter izvede podano funkcijo na vseh elementih seznama in vrne nov pretvorjen seznam. Enostavno lahko napišemo tudi lastno funkcijo, recimo ji `izvedi`.

In [5]:
def izvedi(funk, sez):
    return [funk(x) for x in sez]

print(izvedi(abs, [5,-3,1,9,-7]))

[5, 3, 1, 9, 7]


Da bo funkcija napisana v duhu funkcijskega programiranja, se izognemo for zanki z rekurzijo.

In [6]:
def izvedi(funk, sez):
    return [funk(sez[0])] + izvedi(funk, sez[1:]) if sez else []

print(izvedi(abs, [5,-3,1,9,-7]))

[5, 3, 1, 9, 7]


Podobno deluje funkcija `map`. Edina razlika je, da vrača iterator čez mapirane elemente in ne neposredno seznama, zato ga za bolj jasen izpis pretvorimo v seznam.

In [7]:
print(map(abs, [5,-3,1,9,-7]))
print(list(map(abs, [5,-3,1,9,-7])))

[5, 3, 1, 9, 7]


Map deluje tudi na funkcijah z več argumenti. Ločeno mu podamo sezname prvih, drugih, ... argumentov.

In [8]:
from operator import mul
print(list(map(mul, (3, 5, 1), (2, 1, 3))))

[6, 5, 3]


Če nimamo argumentov primerno ločenih, temveč so grupirani v terke kot v spodnjem primeru, si namesto pretvarjanja v drugačno obliko lahko pomagamo s funkcijo `starmap`.

In [9]:
from itertools import starmap
args = ((3,2), (5,1), (1,3))
print(list(starmap(mul, args)))

[6, 5, 3]


**Filter** je namenjen filtriranju seznama. Sprejme funkcijo in seznam, ter vrne nov seznam elementov, za katere podana funkcija vrne `True`. V spodnjem primeru izluščimo iz seznama vse nenegativne elemente.

In [10]:
print(list(filter(lambda x: x>=0, [5,-3,0,1,-9])))

[5, 0, 1]


**Reduce** nam omogoča, da seznam združimo v končni rezultat. Elemente seznama obdeluje od leve proti desni. S podano funkcijo na vsakem koraku združi že agregiran rezultat (recimo mu akumulator) z naslednjim elementom. Lahko podamo tudi začetno vrednost akumulatorja, sicer se uporabi kar prvi element. Funkcija `reduce` se nahaja v modulu `functools`.

In [11]:
from functools import reduce
print(reduce(mul, [2,4,6,8]))
print(reduce(lambda acc, x: acc + x, [2,4,6,8], 1000.0))

384
1020.0


## Primeri

Uporaba pristopov funkcijskega programiranja zahteva vajo, zato si oglejmo par primerov.

### Group

Z uporabo `reduce` napišimo funkcijo `group`, ki bo sprejela seznam in združila zaporedne enake elemente v pare `(element, pogostost)`. Vrne naj seznam takih parov.

Vrednost akumulatorja `acc` bo grupiran seznam parov že obdelanih elementov seznama. Pri dodajanju novega elementa v akumuliran rezultat moramo obravnavati dva primera. Če gre za element, ki je drugačen od zadnjega, sestavimo nov seznam z dodanim parom `(element, 1)`. Enako storimo na začetku, ko je akumulator prazen. Če pa je nov element enak zadnjemu, moramo popraviti zadnji par. Spreminjati ga ne moremo, lahko pa sestavimo nov seznam z drugačnim zadnjim parom.

In [12]:
def group(sez):
    return reduce(lambda acc, x:
                    acc[:-1] + [(acc[-1][0], acc[-1][1]+1)]
                    if acc and acc[-1][0] == x else
                    acc + [(x,1)],
                  sez,
                  [])

print(group([1,2,2,2,2,3,3,1,3,3,3]))

[(1, 1), (2, 4), (3, 2), (1, 1), (3, 3)]


### Množenje matrik

Napišimo še rešitev v stilu funkcijskega programiranja za množenje matrik. Najprej pa si pripravimo nekaj pomožnih funkcij, ki nam bodo olajšale reševanje.

Skalarni produkt dveh vektorjev (seznamov) lahko napišemo rekurzivno, tako da zmnožimo prva dva elementa in produktu prištejemo skalarni produkt preostanka.

In [13]:
def dot(u, v):
    return u[0]*v[0] + dot(u[1:], v[1:]) if u and v else 0

Namesto rekurzije lahko uporabimo `map` za izračun produktov istoležnih elementov, ki jih nato samo seštejemo s funkcijo `sum`.

In [14]:
def dot(u, v):
    return sum(map(lambda p: p[0]*p[1], zip(u, v)))

Lastno lambda funkcijo v mapu lahko nadomestimo kar s funkcijo `mul`, da dobimo kratko in jasno rešitev.

In [15]:
def dot(u, v):
    return sum(map(mul, u, v))

print(dot([2,4,1], [3,2,1]))

15


Matrike bomo predstavili s seznamom seznamov. Pripravimo si pomožni funkciji `print_vec` in `print_mat`, ki nam bosta olepšali izpis vektorjev in matrik.

In [16]:
A = [[1,2,3,4], 
     [5,6,7,8], 
     [9,10,11,12]]

def print_vec(v, end='\n'):
    print('[' + ''.join(f'{x:4}' for x in v) + ']', end=end)

print_vec(A[0])

[   1   2   3   4]


In [17]:
def print_mat(A):
    for i, v in enumerate(A):
        print('[' if i==0 else ' ', end='')
        print_vec(v, end='')
        print(']' if i==len(A)-1 else '')

print_mat(A)

[[   1   2   3   4]
 [   5   6   7   8]
 [   9  10  11  12]]


Prav nam bosta prišli funkciji `glave` in `repi`, ki bosta matriko razbili na prvi stolpec (glave vrstic) in preostanek matrike (repi vrstic). Lahko bi jih napisali rekurzivno, vendar bo bolj pregledno z uporabo mapa, ker pretvarjamo posamezne vrstice. V primeru glav izbiramo prve elemente, v primeru repov pa seznam brez prvega elementa.

In [18]:
A = [[1,2,3,4], 
     [5,6,7,8], 
     [9,10,11,12]]

def glave(seznami):
    # return [seznami[0][0]] + glave(seznami[1:]) if seznami else []
    return list(map(lambda sez: sez[0], seznami))

print_vec(glave(A))

[   1   5   9]


In [19]:
def repi(seznami):
    return list(map(lambda sez: sez[1:], seznami))

print_mat(repi(A))

[[   2   3   4]
 [   6   7   8]
 [  10  11  12]]


Matrike zaradi načina predstavitve ne moremo enostavno obravnavati po stolpcih. Zato napišimo funkcijo za transpozicijo matrike. V tem primeru se ne bomo mogli izogniti rekurziji. Prva vrstica transponirane matrike je pravzaprav prvi stolpec originalne matrike (glave), preostale vrstice pa dobimo s transpozicijo matrike brez prvega stolpca (repi). Paziti moramo še na robni pogoj. V rekurzivnih klicih matriki odstranjujemo vodilne stolpce, zato ne bomo prišli do praznega seznama, temveč do seznama samih praznih seznamov (vrstic).

In [20]:
def transpose(A):
    return [glave(A)] + transpose(repi(A)) if A[0] else []

print_mat(transpose(A))

[[   1   5   9]
 [   2   6  10]
 [   3   7  11]
 [   4   8  12]]


Prvo vrstico produkta matrik A in B dobimo iz produkta prve vrstice matrike A s stolpci matrike B. Napišimo si pomožno funkcijo, ki bo znala zmnožiti vektor `u` z matriko `B`. Vsak stolpec matrike B (vrstico transponirane matrike) bomo z uporabo map-a množili (`dot`) s podanim vektorjem.

In [21]:
def multiply_vec_mat(u, B):
    return list(map(lambda v: dot(u,v), transpose(B)))

print_vec(multiply_vec_mat([5,6,7,8], A))

[  98 116 134 152]


Končno imamo vse potrebno za množenje matrik. Vsako vrstico matrike A bomo z map-om pretvorili v vrstico rezultata. Pretvorba pa ni nič drugega kot množenje vrstice z matriko B, kar samo si že pripravili.

In [22]:
def multiply(A, B):
    return list(map(lambda u: multiply_vec_mat(u, B), A))
    
A = [[1,2,3,4], 
     [5,6,7,8], 
     [9,10,11,12]]

B = [[5,6],
     [2,1],
     [3,0],
     [1,7]]

print_mat(multiply(A, B))

[[  22  36]
 [  66  92]
 [ 110 148]]


Rešitev lahko tudi razpihnemo z uporabo definicij naših pomožnih funkcij v bolj strašljivo in manj razumljivo obliko. Namen tega je samo demonstrirati, da so gnezdene lambde in mapi povsem sprejemljivi.

In [23]:
def multiply(A, B):
    return list(map(lambda u: list(map(lambda v: sum(map(mul, u, v)), transpose(B))), A))

print_mat(multiply(A, B))

[[  22  36]
 [  66  92]
 [ 110 148]]
